In [54]:
# Initial imports
import pandas as pd
import sklearn.linear_model

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

model_data = (r'C:\Users\clava\data_miners\ML_Model\model_df.csv')

# not working with current code
#pred_data = ('/content/for_predictions.csv')

In [55]:
# Activate functions

# function to turn csv data to datafram
def clean_set(data):
  data_df = pd.read_csv(data)

  data_df['ZIP_CODE'] = data_df['ZIP_CODE'].apply(lambda x: f'{x:05}')

  # set ZIP_CODE as index
  data_df = data_df.set_index('ZIP_CODE')

  data_df['EV_TOTAL'] = data_df.iloc[:, -9:-7].sum(axis=1)

  data_df = data_df[['EV_LEVEL_1', 'EV_LEVEL_2', 'EV_FAST', 'EV_TOTAL',
               'NETWORK_TYPE_ChargePoint','NETWORK_TYPE_Non-Networked','NETWORK_TYPE_Other-Network',
               'TOTAL_HOUSEHOLDS',	'MEDIAN_INCOME',
               'REG_COUNTS']]
  
  return data_df


# function to get target and features
def target_feature(dataframe):
  y = dataframe.REG_COUNTS

  X = dataframe[['EV_LEVEL_1', 'EV_LEVEL_2', 'EV_FAST', 'EV_TOTAL',
                    'NETWORK_TYPE_ChargePoint',	'NETWORK_TYPE_Non-Networked','NETWORK_TYPE_Other-Network',
                    'TOTAL_HOUSEHOLDS',	'MEDIAN_INCOME']]
  
  return (X,y)

# function to generate predictions from dataset
def predict(in_scaler, out_scaler, orig_df, ml_model, data_X):
  data_scaled = in_scaler.transform(data_X)

  pred_scaled = ml_model.predict(data_scaled)

  pred = out_scaler.inverse_transform(pred_scaled)

  pred_df = pd.DataFrame(pred, index=data_X.index, columns=['REG_PREDICT'])

  pred_out_df = orig_df.merge(pred_df, how='left', on='ZIP_CODE')

  return pred_out_df

In [56]:
model_df = clean_set(model_data)

# not working with current code
# pred_df = clean_set(pred_data)
model_df.REG_COUNTS.sum()

1662799

# Train-Test-Split

In [57]:
model_X, model_y = target_feature(model_df)

# not working with current code
# pred_X, pred_y = target_feature(pred_df)

In [58]:
# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.33, random_state=42)

# Scaling

In [59]:
input_scaler = StandardScaler()
output_scaler = StandardScaler()

X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled = input_scaler.transform(X_test)

y_train_scaled = output_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled = output_scaler.transform(y_test.values.reshape(-1,1))

# Linear Regression

In [60]:
# Train the model using the training sets
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

LinearRegression()

#Export Predictions

In [61]:
model_predictions_df = predict(input_scaler, output_scaler, model_df, model, model_X)

# export ev_ml_out_df for Tableau.
model_predictions_df.to_csv('pred_out_df.csv')

# User Interactive

In [65]:
test_df = model_df.copy()

select_feature = input("\n Enter a feature from your dataset: \n EV_LEVEL_1, EV_LEVEL_2, EV_FAST, EV_TOTAL, \n NETWORK_TYPE_ChargePoint, NETWORK_TYPE_Non_Networked, NETWORK_TYPE_Other_Network, \n TOTAL_HOUSEHOLDS, MEDIAN_INCOME \n\n ")

modifier_input = input("Modify feature by what (whole number)? \n")

modifier = int(modifier_input)

# iterate over the dataframe row by row
for index_label, row_series in test_df.iterrows():
    # For each row update the 'Bonus' value to it's double
    test_df.at[index_label , select_feature] = row_series[select_feature] + modifier

pred_X, pred_y = target_feature(test_df)

feature_pred_df = predict(input_scaler, output_scaler, test_df, model, pred_X)

feature_pred_df['TARGET_CHANGE'] = feature_pred_df['REG_PREDICT'] - feature_pred_df['REG_COUNTS']

filename = (select_feature + "_" + str(modifier))

csv_name = "pred_%s.csv" % filename

feature_pred_df.to_csv(csv_name)


feature_pred_df


 Enter a feature from your dataset: 
 EV_LEVEL_1, EV_LEVEL_2, EV_FAST, EV_TOTAL, 
 NETWORK_TYPE_ChargePoint, NETWORK_TYPE_Non_Networked, NETWORK_TYPE_Other_Network, 
 TOTAL_HOUSEHOLDS, MEDIAN_INCOME 

 NETWORK_TYPE_Non-Networked
Modify feature by what (whole number)? 
20


,EV_LEVEL_1,EV_LEVEL_2,EV_FAST,EV_TOTAL,NETWORK_TYPE_ChargePoint,NETWORK_TYPE_Non-Networked,NETWORK_TYPE_Other-Network,TOTAL_HOUSEHOLDS,MEDIAN_INCOME,REG_COUNTS,REG_PREDICT,TARGET_CHANGE
ZIP_CODE,,,,,,,,,,,,
03266,0,2,0,2,0,21,0,851.0,54904.0,0,889.967305,889.967305
07001,0,1,0,1,0,21,0,5022.0,69849.0,173,1360.473829,1187.473829
07003,0,12,5,12,4,21,3,18577.0,78034.0,795,2880.869357,2085.869357
07004,0,6,3,6,0,21,2,2621.0,122063.0,307,2231.712171,1924.712171
07005,0,1,10,1,0,21,1,5489.0,111946.0,445,2176.286921,1731.286921
...,...,...,...,...,...,...,...,...,...,...,...,...
99352,0,13,3,13,1,21,4,12877.0,81410.0,3413,2392.648155,-1020.351845
99354,36,48,0,84,0,65,1,9175.0,73369.0,2082,2943.153453,861.153453
99361,0,1,0,1,0,20,1,750.0,69118.0,100,1270.121444,1170.121444
